In [41]:
import torch
import numpy as np
from listofpathpoint import input_handler
import cnc_input
import copy
import json
def produce_random_data(boundry_recs, randomseed):
    for i in range(0,len(boundry_recs),4):
        np.random.seed(randomseed)
        width = boundry_recs[i+2][0] - boundry_recs[i][0]
        length = boundry_recs[i+2][1] - boundry_recs[i][1]
        original_x,original_y = boundry_recs[i][0].clone(), boundry_recs[i][1].clone()
        points_x = np.random.choice(int(width),2,replace=False)
        points_y = np.random.choice(int(length),2,replace=False)
        points_x.sort()
        points_y.sort()
        boundry_recs[i][0],boundry_recs[i][1] = (original_x + points_x[0]), (original_y + points_y[0])
        boundry_recs[i+1][0],boundry_recs[i+1][1] = (original_x + points_x[1]), (original_y + points_y[0])
        boundry_recs[i+2][0],boundry_recs[i+2][1] = (original_x + points_x[1]), (original_y + points_y[1])
        boundry_recs[i+3][0],boundry_recs[i+3][1] = (original_x + points_x[0]), (original_y + points_y[1])
    return boundry_recs
def random_data_generator(origin_recs, batch_size):
    result_X = torch.Tensor([])
    for seed in range(batch_size):
        temp_recs = origin_recs.clone()
        temp_x = produce_random_data(temp_recs,seed)
        result_X = torch.cat((result_X, temp_x),0)
    result_X = result_X.reshape(batch_size,-1,temp_x.size(1))
    return result_X

test = input_handler('10&15data/25_chips/25_general.json')
X_temp, mask_list_num = test.final_ver_points()
X_temp = torch.FloatTensor(X_temp)
# result = produce_random_data(X_temp,2)
# result_1 = produce_random_data(X_temp,2)
# now we have the produce_random_data, to produce single random data, but we need a batch of data...
X_result = random_data_generator(X_temp, 128)
for map_num in range(128):
    gluewidth =[[354.0,320.0],[354.0,331.0],[387.0,330.0],[387.0,320.0]]
    d = {
    "version": "4.6.0",
    "flags": {},
    "shapes": [{"label":"gluewidth","points":gluewidth}]
    }
    for i in range(25):
        dict = {}
        dict["label"] = str(i)
        dict["points"] = X_result[map_num][:,(0,1)][4*i:4*i+4].tolist()
        d["shapes"].append(dict)

    with open('10&15data/25_chips/25_generate'+str(map_num)+'.json', 'w') as f:
        json.dump(d, f)

tensor([[151.,  30.],
        [197.,  30.],
        [197.,  61.],
        [151.,  61.]])
